In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, StratifiedGroupKFold
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import pickle
import os

In [69]:
df = pd.read_csv('../data/oasis_longitudinal.csv')

# drop Hand (since all R) and MRI ID
df_cleaned = df.drop(columns=['Hand','MRI ID'])
print(df_cleaned.columns)

X = df_cleaned.drop(columns = ['Group', 'Subject ID'])
y = df_cleaned['Group']
subject_ids = df_cleaned['Subject ID'] # groups (for group structure, not labels)

Index(['Subject ID', 'Group', 'Visit', 'MR Delay', 'M/F', 'Age', 'EDUC', 'SES',
       'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF'],
      dtype='object')


In [70]:
# Construct preprocesser (code from eda notebook -- tested it there)
# Decide which encoder to use on each feature
minmax_ftrs = ['Age', 'MMSE', 'EDUC']
onehot_ftrs = ['M/F']
ordinal_ftrs = ['Visit', 'SES', 'CDR']
ordinal_cats = [[1, 2, 3, 4, 5], [-1, 1.0, 2.0, 3.0, 4.0, 5.0], [0.0, 0.5, 1.0, 2.0]]
std_ftrs = ['MR Delay', 'eTIV', 'nWBV', 'ASF']

# Ordinal encoder (separate because need imputer for SES missing values)
ordinal_transformer = Pipeline(steps=[
    ('imputer2', SimpleImputer(strategy='constant',fill_value=-1)),
    ('ordinal', OrdinalEncoder(categories = ordinal_cats))])

# Collect all the encoders
preprocessor = ColumnTransformer(
    transformers=[
        ('minmax', MinMaxScaler(), minmax_ftrs),
        ('onehot', OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore'), onehot_ftrs),
        ('ord', ordinal_transformer, ordinal_ftrs),
        ('std', StandardScaler(), std_ftrs)])

## Pipeline w/ Multivariate Imputation ##
After preprocessing, the only remaining variable with missing values is MMSE, and it only has 2 missing values. Since MMSE is the cognitive test score, I thought that it was reasonable to do multivariate imputation, especially considering how few missing values are in the dataset.

In [71]:
def MLpipe_imp_SGKFold(X, y, preprocessor, ML_algo, param_grid, model_name, eval_metric):
    '''
    Pipeline with:
        Missing values handling method - Multivariate Imputation
        Splitting method -- StratifiedGroupKFold
        Evaluation metric -- Depends on input
    '''

    test_scores = []
    best_models = []

    num_states = 10

    for i in range(num_states):
        # Define random state
        random_state = 29*i
        print(f"---Running random state {random_state}---")

        # ---SPLIT---
        outer_split = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=random_state) # Outer split: split test and other

        for other_idx, test_idx in outer_split.split(X, y, groups=subject_ids):
            break # Break after the first split to separate out the test set for this state
        
        # print(f"\n[Outer Split] Fold {i + 1}")
        # print(f"Train+Validation size: {len(other_idx)}, Test size: {len(test_idx)}")
        # print(f"Train+Validation percentage: {len(other_idx) / len(X) * 100:.2f}%, Test percentage: {len(test_idx) / len(X) * 100:.2f}%")
        # print(f"Groups in Train+Validation: {pd.Series(subject_ids.iloc[other_idx]).nunique()}, Groups in Test: {pd.Series(subject_ids.iloc[test_idx]).nunique()}")

        X_other = X.iloc[other_idx]
        y_other = y.iloc[other_idx]
        subject_ids_other = subject_ids.iloc[other_idx] 
        X_test = X.iloc[test_idx]
        y_test = y.iloc[test_idx]

        kf = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=random_state) # Inner split: 4 fold CV

        # ---PREPROCESS---
        # Main preprocessor is fed into function in "preprocessor"
        final_scaler = StandardScaler()

        # ---IMPUTE---
        imputer = IterativeImputer(estimator = RandomForestRegressor(n_estimators=1), random_state=random_state, max_iter=1000000)

        # ---CONSTRUCT PIPELINE & GRID SEARCH---
        pipeline = make_pipeline(preprocessor, imputer, final_scaler, ML_algo)

        if eval_metric == "accuracy":
            grid = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy', return_train_score=True, n_jobs=-1, verbose=True)
        elif eval_metric == "precision":
            grid = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='precision_macro', return_train_score=True, n_jobs=-1, verbose=True)
        elif eval_metric == "f1_macro":
            grid = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='f1_macro', return_train_score=True, n_jobs=-1, verbose=True)
        elif eval_metric == "f1_weighted":
            grid = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='f1_weighted', return_train_score=True, n_jobs=-1, verbose=True)
        else:
            raise ValueError("Evaluation metric not handled in this pipeline.")
        
        grid.fit(X_other, y_other, groups=subject_ids_other)

        results = pd.DataFrame(grid.cv_results_)
        # print("\nGrid search results:\n", results)

        # ---SAVE BEST MODEL PER RANDOM STATE PER MODEL---
        best_model = grid.best_estimator_
        best_models.append(best_model)
        print('Best model parameters:', grid.best_params_)
        y_test_pred = best_model.predict(X_test)

        if eval_metric == "accuracy":
            best_test_score = accuracy_score(y_test, y_test_pred)
        elif eval_metric == "precision":
            best_test_score = precision_score(y_test, y_test_pred, average="macro")
        elif eval_metric == "f1_macro":
            best_test_score = f1_score(y_test, y_test_pred, average="macro")
        elif eval_metric == "f1_weighted":
            best_test_score = f1_score(y_test, y_test_pred, average="weighted")
        else:
            raise ValueError("Evaluation metric not handled in this pipeline.")
            
        test_scores.append(best_test_score)
        print(f"Test score for random state {29*i}: {best_test_score:.4f}")
        
    # ---SAVE ALL BEST MODELS AND TEST SCORES PER MODEL IN RESULTS---
    file_path = os.path.join('../results/', f'{model_name}_{eval_metric}_results.save')
    with open(file_path, 'wb') as file:
        pickle.dump((best_models, test_scores), file)

    return test_scores, best_models

In [72]:
# Test pipeline on simple logistic regression model first
log_reg = LogisticRegression(solver='saga', max_iter=100000000)
param_grid = {} 
log_reg_test_scores, log_reg_best_models = MLpipe_imp_SGKFold(X, y, preprocessor, log_reg, param_grid, "Test_SimpleLogisticRegression", "accuracy")

---Running random state 0---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 0: 0.9000
---Running random state 29---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 29: 0.8750
---Running random state 58---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 58: 0.9605
---Running random state 87---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 87: 0.9359
---Running random state 116---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 116: 0.8676
---Running random state 145---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 145: 0.9481
---Running random state 174---
Fitting 4 folds for each of 1 candidates, t

In [73]:
# Define model algorithms and parameter grids
models = {
    'SimpleLogisticRegression': (
        LogisticRegression(solver='saga', max_iter=1000000), {} 
    ),
    'L1LogisticRegression': (
        LogisticRegression(penalty='l1', solver='saga', max_iter=1000000),
        {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100]}
    ),
    'L2LogisticRegression': (
        LogisticRegression(penalty='l2', solver='saga', max_iter=1000000),
        {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100]}
    ),
    'ElasticNet': (
        LogisticRegression(penalty='elasticnet', solver='saga', max_iter=100000000),
        {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
         'logisticregression__l1_ratio': [0.001, 0.01, 0.1, 1]}
    ),
    'RandomForestClassifier': (
        RandomForestClassifier(random_state=42),
        {'randomforestclassifier__n_estimators': [100],
         'randomforestclassifier__max_depth': [1, 3, 5, 10, 20, 100],
         'randomforestclassifier__max_features': [0.25, 0.5, 0.75, 1.0, None]}
    ),
    'SupportVectorClassifier': (
        SVC(probability=True),
        {'svc__C': [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3],
         'svc__gamma': [1e-5, 1e-3, 1e-1, 1e1, 1e3, 1e5]}
    ),
    'KNeighborsClassifier': (
        KNeighborsClassifier(),
        {'kneighborsclassifier__n_neighbors': [3, 5, 10, 20],
         'kneighborsclassifier__weights': ['uniform', 'distance']}
    )
}

In [74]:
# Evaluation metric = accuracy
summary = {}
for model_name, (algo, param_grid) in models.items():
    print(f"\nTraining {model_name}...")
    test_scores, best_models = MLpipe_imp_SGKFold(X, y, preprocessor, algo, param_grid, model_name, "accuracy")
    mean_score = np.mean(test_scores)
    stddev_score = np.std(test_scores)
    print("Test Accuracy Scores:", test_scores)
    print(f"Mean Test Accuracy: {mean_score:.4f}")
    print(f"Std Dev of Test Accuracy: {stddev_score:.4f}")
    summary[model_name] = (mean_score, stddev_score)

# Display the summary
print("\nSummary of Results:")
for model_name, (mean_score, stddev_score) in summary.items():
    print(f"{model_name}: Mean Accuracy = {mean_score:.4f}, Std Dev = {stddev_score:.4f}")


Training SimpleLogisticRegression...
---Running random state 0---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 0: 0.9000
---Running random state 29---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 29: 0.8750
---Running random state 58---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 58: 0.9605
---Running random state 87---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 87: 0.9359
---Running random state 116---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 116: 0.8676
---Running random state 145---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 145: 0.9481
---Running random state 174---
Fitti

In [75]:
# Evaluation metric = f1 macro
summary = {}
for model_name, (algo, param_grid) in models.items():
    print(f"\nTraining {model_name}...")
    test_scores, best_models = MLpipe_imp_SGKFold(X, y, preprocessor, algo, param_grid, model_name, "f1_macro")
    mean_score = np.mean(test_scores)
    stddev_score = np.std(test_scores)
    print("Test f1 Scores:", test_scores)
    print(f"Mean Test f1: {mean_score:.4f}")
    print(f"Std Dev of Test f1: {stddev_score:.4f}")
    summary[model_name] = (mean_score, stddev_score)

# Display the summary
print("\nSummary of Results:")
for model_name, (mean_score, stddev_score) in summary.items():
    print(f"{model_name}: Mean f1 = {mean_score:.4f}, Std Dev = {stddev_score:.4f}")


Training SimpleLogisticRegression...
---Running random state 0---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 0: 0.6264
---Running random state 29---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 29: 0.7556
---Running random state 58---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 58: 0.6479
---Running random state 87---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 87: 0.7500
---Running random state 116---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 116: 0.7522
---Running random state 145---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 145: 0.6382
---Running random state 174---
Fitti

In [76]:
# Evaluation metric = f1 weighted
summary = {}
for model_name, (algo, param_grid) in models.items():
    print(f"\nTraining {model_name}...")
    test_scores, best_models = MLpipe_imp_SGKFold(X, y, preprocessor, algo, param_grid, model_name, "f1_weighted")
    mean_score = np.mean(test_scores)
    stddev_score = np.std(test_scores)
    print("Test f1 Scores:", test_scores)
    print(f"Mean Test f1: {mean_score:.4f}")
    print(f"Std Dev of Test f1: {stddev_score:.4f}")
    summary[model_name] = (mean_score, stddev_score)

# Display the summary
print("\nSummary of Results:")
for model_name, (mean_score, stddev_score) in summary.items():
    print(f"{model_name}: Mean f1 = {mean_score:.4f}, Std Dev = {stddev_score:.4f}")


Training SimpleLogisticRegression...
---Running random state 0---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 0: 0.8596
---Running random state 29---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 29: 0.8576
---Running random state 58---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 58: 0.9734
---Running random state 87---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 87: 0.9199
---Running random state 116---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 116: 0.8487
---Running random state 145---
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Best model parameters: {}
Test score for random state 145: 0.9420
---Running random state 174---
Fitti